# Method to remember previous conversation

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are answering chatbot. Please answer the following question in a complete sentence.",
        ),
        # Please use the key 'chat_history' for conversation history without changing it if possible.
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "#Question:\n{question}"),  # Use user input as a variable
    ]
)

# llm 생성
llm = ChatOpenAI(model_name="gpt-4o-mini")

# 일반 Chain 생성
chain = prompt | llm | StrOutputParser()

In [10]:
# Store session
store = {}

# get sesssion based on session id
def get_session_history(session_ids):
    print(f"[conversation sessionId]: {session_ids}")
    if session_ids not in store:  # No session id
        # Create new session and store it
        store[session_ids] = ChatMessageHistory()
    return store[session_ids] 

In [11]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # Function to get session history
    input_messages_key="question",  # Key for user's question to be placed in the template variable
    history_messages_key="chat_history",  # Key for history messages
)

In [12]:
print (store)

{}


In [13]:
chain_with_history.invoke(  
    {"question": "My name is Abby."}, 
    config={"configurable": {"session_id": "abby123"}},
)

[conversation sessionId]: abby123


'Hello, Abby! How can I assist you today?'

In [14]:
print (store)

{'abby123': InMemoryChatMessageHistory(messages=[HumanMessage(content='My name is Abby.', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello, Abby! How can I assist you today?', additional_kwargs={}, response_metadata={})])}


In [15]:
chain_with_history.invoke(  
    {"question": "What is my name?"}, 
    config={"configurable": {"session_id": "abby123"}},
)

[conversation sessionId]: abby123


'Your name is Abby.'

In [16]:
print (store)

{'abby123': InMemoryChatMessageHistory(messages=[HumanMessage(content='My name is Abby.', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello, Abby! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Abby.', additional_kwargs={}, response_metadata={})])}
